In [1]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image
import pandas as pd

/Library/Python/2.7/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
test_dir = 'dataset/processed/test'

n_files = len(os.listdir(test_dir))
n_files

10357

In [3]:
labels_file = 'dataset/labels.csv'


labels_df = pd.read_csv(labels_file)
labels_df = labels_df.sample(frac=1).reset_index(drop=True)


g = labels_df['breed'].value_counts()
classes = list(g.keys())
classes

['scottish_deerhound',
 'maltese_dog',
 'afghan_hound',
 'entlebucher',
 'bernese_mountain_dog',
 'shih-tzu',
 'pomeranian',
 'great_pyrenees',
 'basenji',
 'samoyed',
 'tibetan_terrier',
 'airedale',
 'leonberg',
 'cairn',
 'beagle',
 'japanese_spaniel',
 'blenheim_spaniel',
 'australian_terrier',
 'miniature_pinscher',
 'irish_wolfhound',
 'lakeland_terrier',
 'saluki',
 'papillon',
 'norwegian_elkhound',
 'whippet',
 'siberian_husky',
 'pug',
 'chow',
 'italian_greyhound',
 'pembroke',
 'newfoundland',
 'border_terrier',
 'ibizan_hound',
 'silky_terrier',
 'lhasa',
 'bedlington_terrier',
 'dandie_dinmont',
 'irish_setter',
 'sealyham_terrier',
 'rhodesian_ridgeback',
 'collie',
 'old_english_sheepdog',
 'boston_bull',
 'schipperke',
 'english_foxhound',
 'bouvier_des_flandres',
 'kelpie',
 'african_hunting_dog',
 'bloodhound',
 'bluetick',
 'weimaraner',
 'saint_bernard',
 'labrador_retriever',
 'english_setter',
 'norfolk_terrier',
 'chesapeake_bay_retriever',
 'groenendael',
 'iri

In [4]:
def get_image(filename):
    img = Image.open(filename)
    return np.asarray(img, dtype="int32").flatten() / 255.0

In [5]:
seek_pointer = 0
def get_imgs_and_filenames_batch(size=64):
    global seek_pointer
    continue_flag = True
    start = seek_pointer
    end = seek_pointer = start + size
    imgs = []
    filenames = []
    
    if end >= n_files:
        end = seek_pointer = n_files
        continue_flag = False
    for file_ in os.listdir(test_dir)[start:end]:
        img = get_image(os.path.join(test_dir, file_))
        imgs.append(img)
        filenames.append(file_.split(".")[0])
    print "processed {} out of {}".format(seek_pointer, n_files)
    return imgs, filenames, continue_flag

In [6]:
checkpoint_prefix = 'dataset/tmp/checkpoint-rmsprop/'
graph = tf.get_default_graph()
checkpoint_file = tf.train.latest_checkpoint(checkpoint_prefix)
saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))

In [7]:
seek_pointer = 0
submission_df = pd.read_csv('dataset/submission.csv')
submission_df = submission_df.drop(index=range(10357))
submission_df

with tf.Session(graph=graph) as sess:
    saver.restore(sess, checkpoint_file)
    
    x = graph.get_tensor_by_name("x:0")
    logits_op = graph.get_tensor_by_name("logits/BiasAdd:0")
    
    loop_flag = True
    c = 0
    while loop_flag:
        imgs, filenames, loop_flag = get_imgs_and_filenames_batch()
        logits_ = sess.run(tf.nn.softmax(logits_op), feed_dict={x: imgs})
        new_df = pd.DataFrame(logits_, columns=classes)
        new_df['id'] = filenames
        submission_df = submission_df.append(new_df, ignore_index=True)
        c += 1

INFO:tensorflow:Restoring parameters from dataset/tmp/checkpoint-rmsprop/-2460
processed 64 out of 10357
processed 128 out of 10357
processed 192 out of 10357
processed 256 out of 10357
processed 320 out of 10357
processed 384 out of 10357
processed 448 out of 10357
processed 512 out of 10357
processed 576 out of 10357
processed 640 out of 10357
processed 704 out of 10357
processed 768 out of 10357
processed 832 out of 10357
processed 896 out of 10357
processed 960 out of 10357
processed 1024 out of 10357
processed 1088 out of 10357
processed 1152 out of 10357
processed 1216 out of 10357
processed 1280 out of 10357
processed 1344 out of 10357
processed 1408 out of 10357
processed 1472 out of 10357
processed 1536 out of 10357
processed 1600 out of 10357
processed 1664 out of 10357
processed 1728 out of 10357
processed 1792 out of 10357
processed 1856 out of 10357
processed 1920 out of 10357
processed 1984 out of 10357
processed 2048 out of 10357
processed 2112 out of 10357
processed 217

In [8]:
submission_df.to_csv('dataset/submission_4.csv', index=False)

In [9]:
len(submission_df)

10357